## A simple model for demand and supply of geolocalized services in a city

### Load libraries and classes

In [ ]:
import numpy as np
import pandas as pd

import os
import sys
nb_dir = os.path.dirname(os.getcwd()) ## TODO: find way to put this into some global settings
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg, istat_kpi

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model
from src.models.core import ServiceUnit, ServiceValues, ServiceEvaluator, \
    MappedPositionsFrame, DemandFrame, KPICalculator
from src.models.factories import UnitFactory, SchoolFactory, LibraryFactory 
from src.models.process_tools import GridMaker, ValuesPlotter, JSONWriter
from src.models.city_items import get_random_pos

### Load service data: locations, scales, other info for default city`

In [ ]:
## Load scuole
scuoleFile =  '../data/processed/Milano_datiScuole.csv'
schoolLoader = UnitFactory.createLoader(ServiceType.School, scuoleFile)

# Initialise with a default lengthscale of 0.5 km
schoolUnits = schoolLoader.load(meanRadius=0.5)

In [ ]:
## Load biblioteche
bibliotecheFile =  '../data/processed/Milano_biblioteche.csv'
bibliotecheLoader = UnitFactory.createLoader(ServiceType.Library, bibliotecheFile)

# Initialise with a default lengthscale of 0.5 km
libraryUnits = bibliotecheLoader.load(meanRadius=0.8)

### Sample plots for service levels: evaluate services on a grid (does not account for geolocalized population distrubution)

In [ ]:
# call grid making to discretise service evaluation, this is an alternative to evaluating on the demand units
milanoGridMK = GridMaker({'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'}, gridStep=.6)
testEvaluator = ServiceEvaluator(schoolUnits+libraryUnits)
# call evaluation on internal points only
valuesGrid = testEvaluator.evaluate_services_at(milanoGridMK.grid)

##### Make some plots: plot query locations and service level surfaces

In [ ]:
plotterNew = ValuesPlotter(valuesGrid)
plotterNew.plot_locations()
plotterNew.plot_service_levels(ServiceType.School) 
plotterNew.plot_service_levels(ServiceType.Library)

### Demand import from Censimento Popolazione e Abitazioni (CPA) 2011

In [ ]:
demandData = DemandFrame.create_from_istat_cpa('Milano')

# demo mode, use a portion of the data
demandTest = DemandFrame(demandData.head(500).copy(), False)
calculator = KPICalculator(demandTest, schoolUnits[::10] + libraryUnits[::10], 'Milano')

### Evaluate the services offer at the demand points and average over neighbourhood

In [ ]:
# compute and plot demand/supply interaction for localized services 
calculator.evaluate_services_at_demand() # this might take a while.

calculator.compute_kpi_for_localized_services()

In [ ]:
plotterWeighted = ValuesPlotter(calculator.weightedValues)
plotterWeighted.plot_service_levels(ServiceType.School)

### Append Istat KPI and export averaged values by Neighbourhood-Service-AgeGroup to JSON

In [ ]:
# compute istat kpi as well
calculator.compute_kpi_for_istat_values()

# write KPI to json output
jsonMaker = JSONWriter(calculator)
jsonMaker.write_all_files_to_default_path()